In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC

import sys
sys.path.append("..")
import os

def print_answer(num: int, text: str) -> None:
    if not os.path.exists("answers"):
        os.makedirs("answers")

    with open(os.path.join("answers", f"a{num}.txt"), "w") as f:
        f.write(text)

    print(text)

1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм".

In [2]:
newsgroups = datasets.fetch_20newsgroups(subset="all", categories=["alt.atheism", "sci.space"])
X = newsgroups.data
y = newsgroups.target

2. Вычислите TF-IDF-признаки для всех текстов

Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма

In [3]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам

Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy)

In [4]:
grid = {"C": np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
model = SVC(kernel="linear", random_state=241)
gs = GridSearchCV(model, grid, scoring="accuracy", cv=cv, verbose=1, n_jobs=-1)
gs.fit(X, y)

C = gs.best_params_.get('C')
C

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:   33.4s finished


1.0

4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге

In [5]:
model = SVC(C=C, kernel="linear", random_state=241)
model.fit(X, y)

SVC(kernel='linear', random_state=241)

5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC)

In [6]:
words = np.array(vectorizer.get_feature_names())
word_weights = pd.Series(model.coef_.data, index=words[model.coef_.indices], name="weight")
word_weights.index.name = "word"

top_words = word_weights.abs().sort_values(ascending=False).head(10)
top_words

word
space       2.663165
god         1.920379
atheism     1.254690
atheists    1.249180
moon        1.201611
sky         1.180132
religion    1.139081
bible       1.130612
keith       1.097094
sci         1.029307
Name: weight, dtype: float64

In [7]:
print_answer(1, " ".join(top_words.index.sort_values(ascending=True)))

atheism atheists bible god keith moon religion sci sky space
